In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,621 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,515 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
# Import SparkFiles to handle file management in Spark
from pyspark import SparkFiles

# Define the URL for the home sales dataset stored in AWS S3
# This dataset contains information about home sales including:
# - Price, date, bedrooms, bathrooms, sqft_living, view rating etc.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark's file management system
# This makes the file accessible to all nodes in the Spark cluster
# and allows for distributed processing of the data
spark.sparkContext.addFile(url)


In [4]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)

df.show()

df.createOrReplaceTempView("home_sales")


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

four_bedroom_avg_price = spark.sql("""
    SELECT
        CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) as avg_price,
        YEAR(date) as year
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year
""")

four_bedroom_avg_price.show()


+-----------+----+
|  avg_price|year|
+-----------+----+
|$300,263.70|2019|
|$298,353.78|2020|
|$301,819.44|2021|
|$296,363.88|2022|
+-----------+----+



In [9]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

three_bedroom_three_bathroom_avg_price = spark.sql("""
    SELECT
        CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) as avg_price,
        YEAR(date) as year
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YEAR(date)
    ORDER BY year
""")

three_bedroom_three_bathroom_avg_price.show()

+-----------+----+
|  avg_price|year|
+-----------+----+
|$287,287.82|2019|
|$294,204.16|2020|
|$294,211.46|2021|
|$292,725.69|2022|
+-----------+----+



In [10]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

three_bedroom_three_bathroom_two_floor_avg_price = spark.sql("""
    SELECT
        CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) as avg_price,
        YEAR(date) as year
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(date)
    ORDER BY year
""")

three_bedroom_three_bathroom_two_floor_avg_price.show()

+-----------+----+
|  avg_price|year|
+-----------+----+
|$289,859.14|2019|
|$292,289.09|2020|
|$296,330.96|2021|
|$290,242.99|2022|
+-----------+----+



In [11]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_avg_price = spark.sql("""
    SELECT
        view,
        CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

view_avg_price.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
+----+-------------+
only showing top 20 rows

--- 1.15498948097229 seconds ---


In [12]:
# 7. Cache the temporary table home_sales.

# Caching a table stores it in memory (RAM) rather than disk storage
# This allows for much faster data access and query execution
# Particularly useful for tables that are:
#   - Frequently accessed
#   - Used in multiple queries
#   - Not too large to fit in memory
# The cached table remains in memory until explicitly uncached or the Spark session ends

spark.catalog.cacheTable("home_sales")

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

view_avg_price = spark.sql("""
    SELECT
        view,
        CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

view_avg_price.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
+----+-------------+
only showing top 20 rows

--- 0.9306762218475342 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

#     - Partitioning is a technique to organize data into smaller, more manageable chunks
#     - This can improve query performance, especially for large datasets
#     - Spark will create a separate directory for each partition based on the values in the partition column
#     - This helps in optimizing read/write operations and can speed up queries that filter on the partition column

df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned_parquet")

In [17]:
# 11. Read the parquet formatted data.

home_sales_partitioned_parquet = spark.read.parquet("home_sales_partitioned_parquet")

home_sales_partitioned_parquet.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [18]:
# 12. Create a temporary table for the parquet data.
home_sales_partitioned_parquet.createOrReplaceTempView("home_sales_partitioned")

In [19]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

view_avg_price = spark.sql("""
    SELECT
        view,
        CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

view_avg_price.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|    avg_price|
+----+-------------+
| 100|$1,026,669.50|
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
+----+-------------+
only showing top 20 rows

--- 0.743257999420166 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("home_sales")

In [21]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False